In [ ]:
# -*- coding: utf-8
# Repórter Brasil (https://ruralometro2022.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Procura empresas de deputados federais inscritas em autos de infração trabalhista 
# do antigo Ministério do Trabalho

In [1]:
import pandas as pd
import unidecode

In [2]:
def f(str):
    try:
        return (unidecode.unidecode(str))
    except:
        return (str)

In [3]:
kwargs = {'sep': ';', 'dtype': str, 'encoding': 'utf-8'}
autos_mte = pd.read_csv("originais/autos_mte_10anos.csv", **kwargs)
autos_mte = autos_mte[pd.notnull(autos_mte['cpf_cnpj_infrator'])]
def limpeza1(linha):
    documento = str(linha["cpf_cnpj_infrator"])
    documento = documento.strip()
    documento = documento.replace('.', '')
    documento = documento.replace('-', '')
    documento = documento.replace('/', '')
    return str(documento)
autos_mte["cpf_cnpj_infrator"] = autos_mte.apply(limpeza1, axis=1)
# Preenche zeros a esquerda nos documentos
def compara1(linha):
    documento = str(linha["cpf_cnpj_infrator"])
    
    if len(documento) < 14 and len(documento) >= 12:
        conta = 14 - (len(documento))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + documento
    elif len(documento) == 14:
        alterado = documento
    elif len(documento) < 11:
        conta = 11 - (len(documento))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + documento
    elif len(documento) == 11:
        alterado = documento
    
    return alterado
autos_mte["cpf_cnpj_infrator"] = autos_mte.apply(compara1, axis=1)
def limpeza2(linha):
    documento = str(linha["cpf_cnpj_infrator"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"

autos_mte['cnpj_raiz'] = autos_mte.apply(limpeza2, axis=1)
autos_mte_pf = autos_mte[(autos_mte['cnpj_raiz'] == 'documento_cpf')]
autos_mte_pj = autos_mte[(autos_mte['cnpj_raiz'] != 'documento_cpf')]


In [17]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
empresas_politicos = pd.read_csv("resultados/empresas_deputados/empresas_deputados_fev_2022.csv", **kwargs)

In [18]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
empresas_politicos_socios = pd.read_csv("resultados/empresas_deputados/todos_socios_empresas_deputados_fev_2022.csv", **kwargs)

In [19]:
# Procura empresas de deputados em autos do MTE

In [20]:
empresas_politicos.columns

Index(['nome', 'nome_urna', 'politico_id', 'cpf_mascarado', 'cnpj',
       'cnpj_basico_x', 'nome_socio', 'cnpj_cpf_socio', 'razao_social',
       'nome_fantasia', 'situacao_cadastral', 'cnae_fiscal',
       'cnae_fiscal_secundaria', 'uf', 'municipio'],
      dtype='object')

In [21]:
autos_mte_pj.columns

Index(['num_processo', 'cpf_cnpj_infrator', 'tipo_infracao', 'capitulacao',
       'valor_multa', 'data', 'reincidencia', 'cnpj_raiz'],
      dtype='object')

In [22]:
autos_mte[['cpf_cnpj_infrator']]

,cpf_cnpj_infrator
0,01478313000112
1,76483726001832
2,07352882000101
3,04084451000105
4,07102206000171
...,...
1048570,09622736000176
1048571,27733974000130
1048572,12542573000126
1048573,00831373003804


In [23]:
empresas_mte = pd.merge(empresas_politicos.drop_duplicates('cnpj_basico_x'), autos_mte_pj.drop_duplicates('cnpj_raiz'), left_on='cnpj_basico_x', right_on='cnpj_raiz', how = 'left')

In [24]:
conta = empresas_mte[pd.notnull(empresas_mte['cpf_cnpj_infrator'])].copy()
conta.shape

(35, 23)

In [25]:
conta.to_excel('resultados/empresas_autos_mte/empresas de deputados em autos trabalhistas.xlsx',sheet_name='Sheet1',index=False)

In [26]:
conta.columns

Index(['nome', 'nome_urna', 'politico_id', 'cpf_mascarado', 'cnpj',
       'cnpj_basico_x', 'nome_socio', 'cnpj_cpf_socio', 'razao_social',
       'nome_fantasia', 'situacao_cadastral', 'cnae_fiscal',
       'cnae_fiscal_secundaria', 'uf', 'municipio', 'num_processo',
       'cpf_cnpj_infrator', 'tipo_infracao', 'capitulacao', 'valor_multa',
       'data', 'reincidencia', 'cnpj_raiz'],
      dtype='object')

In [27]:
conta.rename(columns = {'cnpj':'entidade_id'},inplace = True)
conta.rename(columns = {'nome':'politico_nome'},inplace = True)
conta.rename(columns = {'razao_social':'entidade_nome'},inplace = True)

In [28]:
conta = conta[['politico_id', 'entidade_id', 'politico_nome', 'entidade_nome']]

In [29]:
conta.to_csv('resultados/empresas_autos_mte/empresas de deputados em autos trabalhistas.csv',index=False)